In [ ]:
# Import dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
train.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=train, x='LotArea', y='SalePrice');

In [ ]:
train['SaleCondition'].value_counts()

In [ ]:
labels = ['Normal', 'Partial', 'Abnormal', 'Family', 'Alloca', 'AdjLand']

colors = ['gold', 'silver', 'red', 'maroon', 'grey', 'green']

fig, ax = plt.subplots(figsize=(10,5))
ax.pie(train['SaleCondition'].value_counts(), labels=labels, colors=colors,autopct='%1.1f%%', shadow=True,startangle=30)
plt.axis('equal')
plt.show()


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train.head()

## Data Cleaning

In [ ]:
train.isna().sum()

In [ ]:
train.dtypes

In [ ]:
# Create a function , to fill missing values and turn object to numerical

def preprocess_data(df):
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                df[label] = content.fillna(content.median())
        
        if not pd.api.types.is_numeric_dtype(content):
            df[label] = pd.Categorical(content).codes+1
        
    return df

In [ ]:
train = preprocess_data(df=train)
train.head()

In [ ]:
# check again
train.isna().sum()

In [ ]:
train.dtypes

## Split the dataset

In [ ]:
X = train.drop('SalePrice', axis=1)
y= train['SalePrice']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)


## XGBoost

In [ ]:
from xgboost import XGBRegressor

np.random.seed(42)
reg = XGBRegressor()
reg.fit(X_train, y_train)
y_preds = reg.predict(X_val)
y_preds

reg.score(X_val, y_val)

## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)
reg2 = RandomForestRegressor()
reg2.fit(X_train, y_train)
y_preds = reg2.predict(X_val)
y_preds

reg2.score(X_val, y_val)

## GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

np.random.seed(42)
reg3 = GradientBoostingRegressor()
reg3.fit(X_train, y_train)
y_preds = reg3.predict(X_val)
y_preds

reg3.score(X_val, y_val)

## CatBoostRegressor

In [ ]:
from catboost import CatBoostRegressor

np.random.seed(42)
reg4 = CatBoostRegressor(verbose=0)
reg4.fit(X_train, y_train)
y_preds4 = reg4.predict(X_val)
y_preds4

print(reg4.score(X_val, y_val))

## Evaluation metrics

In [ ]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_val, y_preds4, squared=False)
RMSE

## Hyperparmeter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200]}

ideal_model = RandomizedSearchCV(CatBoostRegressor(verbose=0),
                                param_distributions=params,
                                cv=3,
                                n_iter=10,
                                verbose=True)


In [ ]:
ideal_model.fit(X_train, y_train)

In [ ]:
ideal_model.predict(X_val)

In [ ]:
ideal_model.best_params_

In [ ]:
model = CatBoostRegressor(learning_rate=0.2, l2_leaf_reg=5, iterations=1000, depth=6,border_count=32, verbose=0)
model.fit(X_train, y_train)
model.predict(X_val)

## Import Test Dataset

In [ ]:
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
test.head()

## Data Cleaning

In [ ]:
test = preprocess_data(df=test)
test.head()

## Modelling with best model

In [ ]:
model = CatBoostRegressor(learning_rate=0.2, l2_leaf_reg=5, iterations=1000, depth=6,border_count=32, verbose=0)
model.fit(X_train, y_train)
predictions = model.predict(test)

In [ ]:
predictions

In [ ]:
sample = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample

In [ ]:
Submission = pd.DataFrame()
Submission['Id'] = test['Id']
Submission['SalePrice'] = predictions
Submission

In [ ]:
Submission.to_csv('Sample_Submission', index=False)